In [39]:
import pandas as pd
import re
import numpy as np
from collections import Counter
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem.porter import *
from nltk.tokenize import sent_tokenize, word_tokenize
from tqdm import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import euclidean_distances
nltk.download('punkt') 
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
from sklearn.decomposition import NMF
from sklearn.model_selection import train_test_split
from numpy import asarray
from numpy import array
from numpy import zeros
from typing import List
import regex

[nltk_data] Downloading package punkt to /Users/sherryliu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/sherryliu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/sherryliu/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/sherryliu/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
import random
random.seed(10)

In [23]:
df = pd.read_csv('tinder_google_play_reviews.csv', usecols=['at', 'content', 'reviewCreatedVersion'])
df.dropna(subset=['content'],inplace=True)
df.dropna(subset=['reviewCreatedVersion'],inplace=True)

In [51]:
### if we would like to use a bigger dataset, use this code instead
#def get_rows(i):
#    '''input the number of rows you need'''
#    file = open('tinder_google_play_reviews.csv', "r")
#    review = []
#    score = []
#    for i in range(i):
#        line = file.readline()
#        match = regex.search(r'(?<=\,.+\,.+\,)(.+)(?=\,[12345]\,)', line)
#        if match:
#            review.append(match.group())
#            match_s = regex.search(r'(?<=\,.+\,.+\,.+\,)([12345])(?=\,)', line)
#            if match_s:
#                score.append(match_s.group())
#    d = {'content': review, 'score': score}
#    df = pd.DataFrame(d)

,reviews,scores
0,"""It was positively good and I met with lot of ...",5
1,"""Major user policy needed change As a men tryi...",1
2,Fake,1
3,Better,5
4,Not so good,3
...,...,...
994,"""O app não é ruim, mas não entendi pq excluira...",2
995,Good,4
996,"""Dont bother. Itll say """"you got a match"""" and...",1
997,It doesn't unmanned you after awhile,1


In [17]:
df = df.sort_values('at', ascending=False)

In [18]:
df['at'] = pd.to_datetime(df['at'], utc=True)
df['year'] = df['at'].apply(lambda r:r.year)
df = df[df['year'] > 2019]
df.shape

(110770, 4)

In [224]:
df.content = df.content.str.lower()

In [225]:
#regex cleaning
df.content = df.content.str.replace(r"\b\w+n't\b",'not', flags=re.IGNORECASE)
df.content = df.content.str.replace(r'\b(?:awesome|great|excellent|wonderful|well)\b', 'good', flags=re.IGNORECASE)

/var/folders/4f/2_g2k8l95078qdts2g05fkf00000gn/T/ipykernel_1042/4057811400.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df.content = df.content.str.replace(r"\b\w+n't\b",'not', flags=re.IGNORECASE)
/var/folders/4f/2_g2k8l95078qdts2g05fkf00000gn/T/ipykernel_1042/4057811400.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  df.content = df.content.str.replace(r'\b(?:awesome|great|excellent|wonderful|well)\b', 'good', flags=re.IGNORECASE)


In [226]:
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
stopwords = list(stopwords.words('english'))
#we notice from words like app are commonly occuring but dont contribute to the meaning
stopwords = stopwords + ['app', 'apps', 'people','literally','really','very','definitely'] # delete tinder and account since we need them for regex
stopwords.remove('not')
lines = list(df['content'])
cleaned_reviews = []

## Reference: https://gist.github.com/gaurav5430/9fce93759eb2f6b1697883c3782f30de#file-nltk-lemmatize-sentences-py
lemmatizer = WordNetLemmatizer()

# function to convert nltk tag to wordnet tag
def nltk_tag_to_wordnet_tag(nltk_tag):
    if nltk_tag.startswith('J'):
        return wordnet.ADJ
    elif nltk_tag.startswith('V'):
        return wordnet.VERB
    elif nltk_tag.startswith('N'):
        return wordnet.NOUN
    elif nltk_tag.startswith('R'):
        return wordnet.ADV
    else:          
        return None

def lemmatize_sentence(sentence):
    #tokenize the sentence and find the POS tag for each token
    nltk_tagged = nltk.pos_tag(nltk.word_tokenize(sentence))  
    #tuple of (token, wordnet_tag)
    wordnet_tagged = map(lambda x: (x[0], nltk_tag_to_wordnet_tag(x[1])), nltk_tagged)
    lemmatized_sentence = []
    for word, tag in wordnet_tagged:
        if tag is None:
            #if there is no available tag, append the token as is
            lemmatized_sentence.append(word)
        else:        
            #else use the tag to lemmatize the token
            lemmatized_sentence.append(lemmatizer.lemmatize(word, tag))
    return " ".join(lemmatized_sentence)

def lem(sentence):
    '''Intake a list of review, lemmatize them and return a new list'''
    result_lem = []
    for s in tqdm(sentence):
        s_lem = lemmatize_sentence(s)
        result_lem.append(s_lem)
    return result_lem

cleaned_reviews = lem(lines)

100%|█████████████████████████████████| 110770/110770 [01:29<00:00, 1233.40it/s]


In [227]:
df["cleaned_text"] = cleaned_reviews

In [228]:
from textacy.preprocessing.replace import urls, hashtags, numbers, emails, emojis, currency_symbols
df["cleaned_text"] = df.cleaned_text.\
  apply(urls).\
  apply(hashtags).\
  apply(numbers).\
  apply(currency_symbols).\
  apply(emojis).\
  apply(emails)
df.cleaned_text[:5]

0    it be positively good and i meet with lot of n...
1    major user policy need change as a men try the...
3                                                 well
4                                          not so good
5    ton of fake profile and everythings behind a p...
Name: cleaned_text, dtype: object

In [229]:
def regex(line):
    '''regex cleaning for common product issues'''
    line = re.sub(r'\b(ban(?:ned)?|suspend(?:ed)?|b?lock(?:ed)?)\b', '_BAN_', line)
    line = re.sub(r'\b(bug(?:gy)?|crash|server|algorithm)\b', '_BUG_', line)
    line = re.sub(r'\b(regist(?:ration|er|ered))\b', '_REGISTER_', line)
    line = re.sub(r'\b(log(?:ging|ged)?(?:-| )?in)\b', '_LOGIN_', line)
    line = re.sub(r'\b(faked?|bot|not real)\b','_FAKE_', line)
    line = re.sub(r'\b(match(?:ing)?)\b','_MATCHING_', line)
    line = re.sub(r'\b(location|distance)\b','_LOCATION_', line)
    line = re.sub(r'\b(subscript(?:ion)?|charg(?:ed|ing|es)?|pay(?: )?walls?|expensive|premium|in-app-purchase|tinder gold|membership|payment)\b','_CHARGES_', line)
    line = re.sub(r'\b(notification|spam(?:med)?|promotion)\b','_SPAM_', line)
    line = re.sub(r'\b(scam(?:mer|med)?)\b','_SCAM_', line)
    line = re.sub(r'\bsuper likes?\b','_SUPER_LIKE_', line)
    line = re.sub(r'\blikes?\b','_LIKE_', line)
    line = re.sub(r'\brewinds?\b','_REWIND_', line)
    line = re.sub(r'\bpassport\b','_PASSPORT_', line)
    line = re.sub(r'\bmessages?\b','_MESSAGE_', line)
    line = re.sub(r'\bswipe(s| left| right)?\b','_SWIPE_', line)
    line = re.sub(r'\btop picks?\b','_TOP_PICK_', line)
    return line

In [230]:
review_all = df['cleaned_text'].tolist()
regex_1 = [regex(r) for r in review_all]
regex_1[:5]

['it be positively good and i meet with lot of nice , interesting , fun , loveable people , but there be a few pervert who on profile be not what they claim to be .',
 'major user policy need change as a men try the apps , let me tell you it have full of broken soul , become an instagram follower mining , lot of wahmen will try to _SCAM_ you ... and as good not to mention the professional _SCAM_ that exist in most dating apps . if tinder still want to be relevant in these climate , major policy / user interaction need change . it have become a business platform for majority of the oposite sex . when apps not longer serve it purpose , it will die .',
 'well',
 'not so good',
 'ton of _FAKE_ profile and everythings behind a _CHARGES_ . why would anyone be foolish enough to pay for a date app with so many _FAKE_ profile .']

In [231]:
def get_tfidf_vec(review, i):
    '''Intake a list of review and return tf-idf report'''
    vectorizer = TfidfVectorizer(ngram_range=(i,i),
                             token_pattern=r'\b[a-zA-Z\_]{3,}\b',
                             max_df=0.4, stop_words=stopwords, max_features=1000, binary=True)
    X = vectorizer.fit_transform(review)
    terms = vectorizer.get_feature_names()
    tf_idf = pd.DataFrame(X.toarray().transpose(), index=terms)
    tf_idf_sum = tf_idf.sum(axis=1)
    score = pd.DataFrame(tf_idf_sum, columns=["score"])
    score.sort_values(by="score", ascending=False, inplace=True)
    nmf = NMF(n_components=4)
    W = nmf.fit_transform(X)
    H = nmf.components_
    tf_idf_topic = pd.DataFrame(X.toarray(), columns=terms)
    def get_top_tf_idf_tokens_for_topic(H: np.array, feature_names: List[str], num_top_tokens: int = 5):
        for topic, vector in enumerate(H):
            print(f"TOPIC {topic}\n")
            total = vector.sum()
            top_scores = vector.argsort()[::-1][:num_top_tokens]
            token_names = list(map(lambda idx: feature_names[idx], top_scores))
            strengths = list(map(lambda idx: vector[idx] / total, top_scores))
            for strength, token_name in zip(strengths, token_names):
                print(f"\b{token_name} ({round(strength * 100, 1)}%)\n")
            print(f"=" * 50)
    get_top_tf_idf_tokens_for_topic(H, tf_idf_topic.columns.tolist(), 5)
    return W, H, tf_idf.transpose(), score

In [232]:
df['regex_1'] = regex_1
review_good = df[df['score'] >= 4]['regex_1']
review_bad = df[df['score'] < 4]['regex_1']

In [233]:
W, H, tf_idf_bad, score_bad = get_tfidf_vec(review_bad, 3)
score_bad[:20]

/Users/sherryliu/opt/anaconda3/lib/python3.9/site-packages/sklearn/decomposition/_nmf.py:312: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  warnings.warn(("The 'init' value, when 'init=None' and "


TOPIC 0

get _ban_ reason (46.1%)

account get _ban_ (12.0%)

_ban_ reason not (3.8%)

keep get _ban_ (1.3%)

reason not even (0.9%)

TOPIC 1

not waste time (34.7%)

waste time money (27.4%)

dont waste time (1.3%)

please not waste (0.8%)

complete waste time (0.7%)

TOPIC 2

many _fake_ profile (67.2%)

way many _fake_ (3.3%)

_fake_ profile _scam_ (1.9%)

_fake_ profile _fake_ (1.9%)

_fake_ profile not (1.6%)

TOPIC 3

not get _matching_ (50.9%)

get _matching_ not (2.5%)

still not get (1.7%)

get _matching_ even (1.6%)

get _matching_ unless (1.3%)



,score
get _ban_ reason,443.109855
not waste time,318.406177
account get _ban_,297.933534
many _fake_ profile,296.490275
not get _matching_,291.554996
waste time money,287.460441
_number_ mile away,275.007478
account _ban_ reason,213.860982
pay see _like_,207.275449
got _ban_ reason,204.497896


In [234]:
def get_top_documents_for_each_topic(W: np.array, documents: List[str], num_docs: int = 10):
    sorted_docs = W.argsort(axis=0)[::-1]
    top_docs = sorted_docs[:num_docs].T
    per_document_totals = W.sum(axis=1)
    for topic, top_documents_for_topic in enumerate(top_docs):
        print(f"Topic {topic}")
        for doc in top_documents_for_topic:
            score = W[doc][topic]
            percent_about_topic = round(score / per_document_totals[doc] * 100, 1)
            print(f"{percent_about_topic}%")
            print(documents[doc].replace('\n',''))
        print("=" * 50)

In [235]:
get_top_documents_for_each_topic(W, review_bad.tolist(), num_docs=10)

Topic 0
100.0%
i get _BAN_ for no reason lmao
100.0%
i get _BAN_ for no reason , please tell me how i can fix this issue _EMOJI__EMOJI_
100.0%
i get _BAN_ for no reason . this app be stupid
100.0%
bore get _BAN_ for no reason too
100.0%
i get _BAN_ for no reason . i be nice to everyone .
100.0%
good do , tinder ! i have undergone account verification two day ago and today i get _BAN_ for no reason . what a shame ! i not a tinder swindler ! i will keep my rating _NUMBER_ until my account recover . shame on you !
100.0%
literally get _BAN_ for no reason
100.0%
i get _BAN_ for no reason what so ever_EMOJI_
100.0%
i get _BAN_ for no reason .
100.0%
i get _BAN_ for no reason .
Topic 1
100.0%
not waste your time or money on this app . customer support guy name larry be a milked teaspoon . app and support full of bellends
100.0%
every female on this app be either an escort ( i 've have _NUMBER_ escort _MATCHING_ with me in one day ) or a scamer ( i be stupid and not realize until it be too la

In [26]:
def regex_2(line):
    '''additional regex cleaning based on previous topic modeling result'''
    line = re.sub(r'\b(\_emoji\_)\b', '', line)
    line = re.sub(r'\b\_FAKE\_ (?:account|profile)\b', '_FAKE_', line)
    line = re.sub(r'\bwaste(?: time| money)?\b', '_WASTE_', line)
    line = re.sub(r'\b(phone number)\b', '_PHONE_', line)
    line = re.sub(r'\bget \_MATCHING\_\b', '_MATCHING_', line)
    line = re.sub(r'\b(customer service)\b','_CUSTOMER_SERVICE_', line)
    line = re.sub(r'\b(get _BAN_)\b','_BAN_', line)
    return line

In [237]:
review_all = df['regex_1'].tolist()
regex_2 = [regex_2(r) for r in review_all]
regex_2[:5]

['it be positively good and i meet with lot of nice , interesting , fun , loveable people , but there be a few pervert who on profile be not what they claim to be .',
 'major user policy need change as a men try the apps , let me tell you it have full of broken soul , become an instagram follower mining , lot of wahmen will try to _SCAM_ you ... and as good not to mention the professional _SCAM_ that exist in most dating apps . if tinder still want to be relevant in these climate , major policy / user interaction need change . it have become a business platform for majority of the oposite sex . when apps not longer serve it purpose , it will die .',
 'well',
 'not so good',
 'ton of _FAKE_ and everythings behind a _CHARGES_ . why would anyone be foolish enough to pay for a date app with so many _FAKE_ .']

In [238]:
df['regex_2'] = regex_2
review_good_2 = df[df['score'] >= 4]['regex_2']
review_bad_2 = df[df['score'] < 4]['regex_2']

In [239]:
W, H, tf_idf_bad, score_bad_2 = get_tfidf_vec(review_bad_2, 3)
score_bad_2[:20]

/Users/sherryliu/opt/anaconda3/lib/python3.9/site-packages/sklearn/decomposition/_nmf.py:312: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  warnings.warn(("The 'init' value, when 'init=None' and "


TOPIC 0

not _waste_ time (58.8%)

_ban_ without reason (1.2%)

_fake_ not _waste_ (1.1%)

please not _waste_ (1.1%)

_waste_ time not (0.8%)

TOPIC 1

account _ban_ reason (52.8%)

_ban_ reason not (6.4%)

_ban_ reason give (1.8%)

say account _ban_ (1.4%)

got _ban_ reason (1.2%)

TOPIC 2

_number_ mile away (37.5%)

show _number_ mile (4.3%)

_number_ _number_ mile (3.6%)

_matching_ _number_ mile (3.5%)

set _number_ mile (2.8%)

TOPIC 3

_waste_ time money (63.0%)

dont _waste_ time (3.6%)

complete _waste_ time (1.8%)

time money not (1.6%)

total _waste_ time (1.5%)



,score
not _waste_ time,309.422346
account _ban_ reason,294.387529
_waste_ time money,279.295139
_number_ mile away,278.928677
not get _matching_,211.434684
got _ban_ reason,206.943633
pay see _like_,205.953549
_ban_ account reason,195.905838
_ban_ reason not,178.980233
give _number_ star,173.808977


In [240]:
get_top_documents_for_each_topic(W, review_bad.tolist(), num_docs=5)

Topic 0
100.0%
use hinge if your look to meet people . not waste your time with this poorly make app . the owner be a hellspawn ....
100.0%
tinder be full of _FAKE_ profile , _SPAM_ and totally rig to take your money , for practically nothing . absolute rubbish not waste your time guy 's ! !
100.0%
use to be good . not anymore , not waste your time instal it .
100.0%
the only people leave on this app be _FAKE_ and only fan catfishing account , not waste your time ...
100.0%
this be an ego-boosting app for girl . nothing more . not waste your time
Topic 1
100.0%
account be _BAN_ for no reason
100.0%
my account get _BAN_ with out any reason .
100.0%
my account get _BAN_ for no reason only have it a day
100.0%
my account get _BAN_ for no reason . i love tinder .
100.0%
account get _BAN_ for no reason
Topic 2
100.0%
every time they change their _CHARGES_ tier it just take stuff away from the bottom tier to force you to get the next one . literally take _NUMBER_ thing away from the bottom a

In [241]:
df_bad = review_bad_2.to_frame()
df_bad.shape

(67806, 1)

In [242]:
df_bad['ban'] = df_bad['regex_2'].apply(lambda x: 1 if x.find('_BAN_') != -1 else 0)
df_ban = df_bad[df_bad['ban'] == 1]
ban = df_ban.regex_2.tolist()
ban = [re.sub(r'\b\_BAN\_\b', '', line) for line in ban]

In [243]:
def get_tfidf_vec_sec(review, i):
    '''Intake a list of review and return tf-idf report'''
    vectorizer = TfidfVectorizer(ngram_range=(i,i),
                             token_pattern=r'\b[a-zA-Z\_]{3,}\b',
                             max_df=0.4, stop_words=stopwords, max_features=1000, binary=True)
    X = vectorizer.fit_transform(review)
    terms = vectorizer.get_feature_names()
    tf_idf = pd.DataFrame(X.toarray().transpose(), index=terms)
    tf_idf_sum = tf_idf.sum(axis=1)
    score = pd.DataFrame(tf_idf_sum, columns=["score"])
    score.sort_values(by="score", ascending=False, inplace=True)
    return tf_idf.transpose(), score

In [244]:
tf_idf_ban, score_ban = get_tfidf_vec_sec(ban, 2)
score_ban[:20]

,score
account reason,322.233003
not even,261.398554
reason not,192.760472
without reason,163.665721
got reason,163.509281
account not,158.551520
not know,144.443771
appeal process,127.944682
not use,124.954271
_number_ year,122.451651
